In [1]:
import pandas as pd 
import numpy as np
import csv 
from tqdm.notebook import tqdm_notebook
from deep_translator import GoogleTranslator
from langdetect import detect
tqdm_notebook.pandas()
import cupy,cuml,cudf
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
import time
import os
import re
from bs4 import BeautifulSoup

/home/shisuimadara/.local/lib/python3.10/site-packages/cupy/_environment.py:440: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda102, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [5]:
translator = GoogleTranslator(source='auto', target='en')

def translate(sentence):
    try:
        lang =detect(sentence)
        if lang == 'en':
            return sentence
        else:
            x = translator.translate(str(sentence))
            return x
    except:
        return sentence

In [6]:
def doTranslate(path_ls):
  
  for i,path in enumerate(path_ls):
      df = pd.read_csv(path)
      df["final"] = df["final"].progress_apply(translate)
      # df["BULLET_POINTS"] = df["BULLET_POINTS"].progress_apply(translate)
      # df["DESCRIPTION"] = df["DESCRIPTION"].progress_apply(translate)
      # if i==0:
      #   df.to_csv(f"/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/train_out.csv",index=False)
      # else:
      df.to_csv(f"/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/test4_out.csv",index=False)

In [9]:
def preprocessing(phrase):
     # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def preprocess(sentence):
    sentence = str(sentence)
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = preprocessing(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    html_pattern = re.compile('<.*?>')
    sentence = html_pattern.sub(r'', sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    
    return sentence

def do_preprocessing (path_ls):
  for i,path in enumerate(path_ls):
      df = pd.read_csv(path)
      df["final"] = df["final"].progress_apply(preprocess)
      df.to_csv(f"/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/test_preprocess.csv",index=False)

In [6]:
import math
def create_embedding (sentence):
  model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')
  if type(sentence) == type(""):
    embeddings = model.encode(sentence)
    return embeddings
  if math.isnan(sentence):
    sentence = ""
    embeddings = model.encode(sentence)
    return embeddings
  else:
    li = []
    for x in sentence:
      print(sentence)
      li.append(model.encode(x))
      return li
  # print(embeddings)

def do_embedding (path_ls):
  for i,path in enumerate(path_ls):
      df = pd.read_csv(path)
      df["final"] = df["final"].progress_apply(create_embedding)
      df.to_csv(f"/content/drive/MyDrive/datasetb2d9982/dataset/a_embed_out.csv",index=False)



In [5]:
train_path = "/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/train.csv"
test_path = "/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
test_df['final'] = test_df['TITLE'] + " " + test_df['BULLET_POINTS'] + " " + test_df['DESCRIPTION']
train_df['final'] = train_df['TITLE'] + " " + train_df['BULLET_POINTS'] + " " + train_df['DESCRIPTION']

In [6]:

train_df["final"].fillna("No Data", inplace = True)

train_df['final'].isnull().sum()

test_df["final"].fillna("No Data", inplace = True)

test_df['final'].isnull().sum()
train_df = train_df.drop(['PRODUCT_ID', 'BULLET_POINTS', 'DESCRIPTION','TITLE'], axis=1)
test_df = test_df.drop(['PRODUCT_ID', 'BULLET_POINTS', 'DESCRIPTION', 'TITLE'], axis=1)
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [10]:
path_ls = [ "/home/shisuimadara/Amazon-ML-Challenge2021/scripts/test.csv"]

doTranslate(path_ls)

path_after_translate = [ "/home/shisuimadara/Amazon-ML-Challenge2021/scripts/test.csv"]
do_preprocessing(path_after_translate)


train_df_cpu = pd.read_csv(train_path,escapechar="\\",quoting=csv.QUOTE_NONE)
test_df_cpu = pd.read_csv(test_path,escapechar="\\",quoting=csv.QUOTE_NONE)
test_df_cpu["PRODUCT_TYPE_ID"].fillna(0, inplace = True)
train_df_cpu["PRODUCT_TYPE_ID"].fillna(0, inplace = True)

  0%|          | 0/734736 [00:00<?, ?it/s]

/tmp/ipykernel_178120/2520623819.py:36: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentence = BeautifulSoup(sentence, 'lxml').get_text()


In [ ]:
model = TfidfVectorizer(stop_words='english', binary=True, max_features=128)
train_embed = model.fit_transform(train_df.final).toarray()
test_embed = model.transform(test_df.final).toarray()
BATCH_SIZE = 256
test_pred = []
NUM_BATCHES = test_embed.shape[0]//BATCH_SIZE

start = time.time()
for i in range(NUM_BATCHES):
    start = i*BATCH_SIZE
    end = (i+1)*BATCH_SIZE
    sim = cupy.matmul(train_embed,test_embed[start:end].T).T
    
    for j in range(end-start):
        ids = cupy.where(sim[j,] > 0.4)[0]
        
        ls = train_df_cpu.iloc[cupy.asnumpy(ids)]["PRODUCT_TYPE_ID"]
        if math.isnan(ls.mean()):
            test_pred.append(0)

        else:
            test_pred.append(ls.mean())
        # print(int(ls.mean()))
        # # print(val)
        # test_pred.append(ls)
    if(i % 50 == 0):
        print("DONE PRODUCTS :", end)
        print("Time : ",int(time.time()-start),"sec")
print("COSINE SIMILARITY")

test_pred


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20000, random_state=2500, n_init="auto").fit(test_pred[:])



0
DONE PRODUCTS : 256
Time :  1682228974 sec
1
DONE PRODUCTS : 512
Time :  1682228718 sec


In [ ]:
a = cudf.DataFrame(data=train_embed)
x = kmeans.predict(a.to_numpy())
df2 = pd.read_csv('/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/train.csv')
first_ = df2['PRODUCT_ID']
first = train_df['PRODUCT_TYPE_ID']
x_ = pd.Series(x)
df_ = pd.DataFrame({'PRODUCT_ID': first_, 'PRODUCT_TYPE_ID': first, 'id': x_})
df_.to_csv('nn_train_data.csv', index = False)

In [ ]:
a = cudf.DataFrame(data=train_embed)
x = kmeans.predict(a.to_numpy())
df2 = pd.read_csv('/home/shisuimadara/Amazon-ML-Challenge2021/datasetb2d9982/dataset/test.csv')
first_ = df2['PRODUCT_ID']
first = train_df['PRODUCT_TYPE_ID']
x_ = pd.Series(x)
df_ = pd.DataFrame({'PRODUCT_ID': first_, 'PRODUCT_TYPE_ID': first, 'id': x_})
df_.to_csv('nn_test_data.csv', index = False)

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import pandas as pd

# load training data from a CSV file
# load training data from a CSV file
df = pd.read_csv('nn_train_data.csv')
# create some sample data
product_type_id = df['PRODUCT_TYPE_ID']
product_type_id.fillna(0, inplace=True)
cluster_id = df['id']
cluster_id.fillna(0, inplace=True)
X_train = np.stack([product_type_id, cluster_id], axis=-1)
product_length = df['PRODUCT_LENGTH']
product_length.fillna(0, inplace=True)
y_train = np.array(product_length)

# create an SVM regression model with a radial basis function kernel
svm = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.2)

# train the model
svm.fit(X_train, y_train)

# make some predictions on the training data and compute the mean squared error
y_pred = svm.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print(f"Mean squared error on the training data: {mse:.2f}")



print(y_pred)

[2492.56,
 1639.0,
 3076.375,
 3076.375,
 2492.56,
 3076.375,
 2492.56,
 2492.56,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 8470.0,
 2492.56,
 12556.0,
 2492.56,
 2492.56,
 2492.56,
 2492.56,
 1875.0,
 3076.375,
 3076.375,
 2492.56,
 3076.375,
 3076.375,
 3076.375,
 1383.0,
 7284.0,
 2492.56,
 7284.0,
 3076.375,
 2492.56,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 3076.375,
 2834.0,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 6548.0,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 3076.375,
 2492.56,
 2492.56,
 2492.56,
 2918.0,
 2492.56,
 11192.0,
 2492.56,
 2911.0,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 3076.375,
 2492.56,
 2492.56,
 3076.375,
 9701.0,
 6548.0,
 2492.56,
 3076.375,
 3076.375,
 12680.0,
 2492.56,
 3076.375,
 2492.56,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 2211.0,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 3076.375,
 2492.56,
 3076.375,
 3076.375,
 2492.56,
 2492.56,
 3076.375,
 2492.56,
 2492

In [ ]:
# make some predictions on new data
X_test = pd.read_csv('nn_test_1.csv')
product_type_id = X_test['PRODUCT_TYPE_ID']
product_type_id.fillna(0, inplace=True)
cluster_id = X_test['id']
cluster_id.fillna(0, inplace=True)
X_test = np.stack([product_type_id, cluster_id], axis=-1)
y_pred = svm.predict(X_test)

In [ ]:
X_test = pd.read_csv('nn_test_1.csv')
X_test.head()

In [ ]:
final_df = pd.DataFrame()
final_df['PRODUCT_ID'] = X_test['PRODUCT_ID']
final_df['PRODUCT_LENGTH'] = y_pred
final_df.head()
final_df.to_csv('sub0.csv', index=False)